In [1]:
from datetime import date
from data_manager import DataManager
from scipy.stats import median_abs_deviation
import pandas as pd


In [2]:
self = DataManager()

def get_average_team_stat(self, df, last_n_team_games):
    teams = self.query_teams()
    
    # Initialize columns outside the loop
    stats = ['fgm', 'fga', 'fg3m', 'fg3a', 'points', 'rebounds', 'assists', 'steals', 'blocks', "to"]
    for stat in stats:
        df.loc[:, f'team_{stat}_mad'] = None
        df.loc[:, f'team_{stat}_average'] = None

    for team in df['TEAM'].unique():
        team_id = [team_object.id for team_object in teams if team_object.nickname == team][0]
        print(team_id)
        team_games = self.get_and_save_team_data(team_id).sort_values(by='date', ascending=False).head(last_n_team_games)
        print(team_games.columns)
        for stat in stats:
            total_points_average = team_games[stat].mean()
            mad_points = median_abs_deviation(team_games[stat], scale="normal")
            df.loc[df['TEAM'] == team, f'team_{stat}_average'] = total_points_average
            df.loc[df['TEAM'] == team, f'team_{stat}_mad'] = mad_points
            
    return df

In [3]:

analyzed_props = self.get_analyzed_props()
analyzed_prop_records = pd.DataFrame.from_dict([prop.entry for prop in analyzed_props])
profitable_props = analyzed_prop_records[analyzed_prop_records['EV'] > 0]


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
          player_name       team      stat  over_threshold  over_odds  \
0   Jaren Jackson Jr.  Grizzlies    points            19.5       -125   
1        Santi Aldama  Grizzlies    points            11.5       -125   
2           Zach Edey  Grizzlies    points             8.5       -105   
3        Jaylen Wells  Grizzlies    points            11.5       -120   
4           Ja Morant  Grizzlies    points            22.5       -130   
5     Kelly Oubre Jr.      76ers    points            18.5       -105   
6          Kyle Lowry      76ers    points             6.5        100   
7        Caleb Martin      76ers    points            12.5       -110   
8      Andre Drummond      76ers    points            10.5       -125   
9        Tyrese Maxey      76ers    points            30.5       -110   
10       Santi Aldama  Grizzlies   assists             2.5       -120

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
view = profitable_props[['PLAYER', 'TEAM', 'STAT', 'TYPE']]
last_n_team_games = 5
df = get_average_team_stat(self, profitable_props, last_n_team_games=last_n_team_games)

27
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date', 'game_id', 'pace', 'def_rating', 'e_def_rating', 'off_rating',
       'e_off_rating'],
      dtype='object')
19
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date', 'game_id', 'pace', 'def_rating', 'e_def_rating', 'off_rating',
       'e_off_rating'],
      dtype='object')


C:\Users\rusta\AppData\Local\Temp\ipykernel_8696\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_mad'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_8696\4244349758.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_average'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_8696\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [5]:
print(df) 

df = df.sort_values(by="EV", ascending=False).reset_index(drop=True)
df.to_csv("temp.csv")
print(df.columns)


               PLAYER       TEAM      STAT  THRESH  ODDS   TYPE      PROB  \
0   Jaren Jackson Jr.  Grizzlies    points    19.5  -125   over  0.762286   
4           Zach Edey  Grizzlies    points     8.5  -105   over  0.544347   
7        Jaylen Wells  Grizzlies    points    11.5  -110  under  0.751990   
11    Kelly Oubre Jr.      76ers    points    18.5  -130  under  0.625614   
12         Kyle Lowry      76ers    points     6.5   100   over  0.656443   
15       Caleb Martin      76ers    points    12.5  -120  under  0.806474   
17     Andre Drummond      76ers    points    10.5  -105  under  0.729423   
19       Tyrese Maxey      76ers    points    30.5  -120  under  0.717034   
23       Jaylen Wells  Grizzlies   assists     1.5   105  under  0.503668   
25          Zach Edey  Grizzlies   assists     0.5   135  under  0.716531   
26  Jaren Jackson Jr.  Grizzlies   assists     2.5   135   over  0.520957   
31     Andre Drummond      76ers   assists     0.5  -120  under  0.778801   

In [6]:
from collections import deque

teams = df['TEAM'].unique()
if len(teams) != 2:
    raise ValueError("The TEAM column must contain exactly two unique values for this operation.")

# Get the values to be swapped
value1 = df.loc[df['TEAM'] == teams[0], 'team_to_average'].values[0]
value2 = df.loc[df['TEAM'] == teams[1], 'team_to_average'].values[0]

# Create a mapping to swap the values
mapping = {teams[0]: value2, teams[1]: value1}

# Apply the mapping to the 'team_average_to' column
df['team_to_average'] = df['TEAM'].map(mapping)

# Rename the column
df.rename(columns={'team_to_average': 'oppteam_to_average'}, inplace=True)

    

In [7]:
team_stats_cols = [col for col in df.columns if "average" in col]
team_stats = df.copy()[team_stats_cols]
new_cols = [col.split("_")[1] for col in team_stats.columns]
team_stats.columns = new_cols
team_stats['TEAM'] = df['TEAM']


initial_availability = {}
for team in team_stats['TEAM'].unique():
    initial_availability[team] = dict(team_stats[team_stats['TEAM'] == team].reset_index(drop=True).iloc[0, :])



                

In [8]:

import copy

parlay_length = 8
parlay_count = 10
used = []
parlays = []
for i in range(parlay_count):
    availability = copy.deepcopy(initial_availability)
    parlay = []
    counter = 0
    for i, row in df.iterrows():
        if len(parlay) == parlay_length:
            break
        if i in used:
            continue
        else:
            if row['STAT'] in new_cols:
                stat = row['STAT']
                if row['TYPE'] == "over" and availability[row['TEAM']][stat] - (row['THRESH'] * 3) > 0:
                    availability[row['TEAM']][stat] -= row['THRESH']
                    parlay.append(i)
                    used.append(i)
                elif row['TYPE'] == "under" and availability[row['TEAM']][stat] + row['THRESH'] < initial_availability[row['TEAM']][stat]:
                    availability[row['TEAM']][stat] += row['THRESH']
                    parlay.append(i)
                    used.append(i)
    print(parlay)
    parlays.append(parlay)
parlays = [parlay for parlay in parlays if len(parlay) == parlay_length]
print(parlays)


                    

[7, 11, 13, 14, 16, 17, 19, 20]
[26, 27, 29, 30, 32, 37]
[]
[]
[]
[]
[]
[]
[]
[]
[[7, 11, 13, 14, 16, 17, 19, 20]]


In [9]:
parlay_dfs = []
# Segment the original DataFrame
for indexes in parlays:
    parlay_df = df.iloc[indexes]
    parlay_dfs.append(parlay_df)

# Display the resulting DataFrames
for i, segment_df in enumerate(parlay_dfs):
    print(f"Parlay {i}:\n{segment_df}\n")
    segment_df.to_csv(f"parlays/{date.today()}_{i}_greedy.csv")

Parlay 0:
               PLAYER       TEAM      STAT  THRESH  ODDS   TYPE      PROB  \
7   Jaren Jackson Jr.  Grizzlies    points    19.5  -125   over  0.762286   
11         Kyle Lowry      76ers    points     6.5   100   over  0.656443   
13         Kyle Lowry      76ers   assists     2.5  -140   over  0.749939   
14         Kyle Lowry      76ers  rebounds     2.5   140   over  0.530546   
16  Jaren Jackson Jr.  Grizzlies       fgm     6.5  -135   over  0.719616   
17       Jaylen Wells  Grizzlies       fgm     4.5  -155  under  0.756494   
19  Jaren Jackson Jr.  Grizzlies   assists     2.5   135   over  0.520957   
20  Jaren Jackson Jr.  Grizzlies      fg3m     1.5   125   over  0.537163   

          EV  HOUSE_PROB team_fgm_mad team_fgm_average team_fga_mad  \
7   1.860577    0.555556     7.413011             43.2     7.413011   
11  1.564433    0.500000     5.930409             36.6    16.308624   
13  1.428046    0.583333     5.930409             36.6    16.308624   
14  1.366556